In [2]:
import pandas as pd
import numpy as np
from subprocess import check_output
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import Imputer
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [3]:
X = pd.read_csv('Clustered Data');

In [4]:
y= X[['Power Outage']];
X = X.drop(['Power Outage'], axis=1);

In [5]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

class ReduceVIF(BaseEstimator, TransformerMixin):
    def __init__(self, thresh=5.0, impute=True, impute_strategy='median'):
        # From looking at documentation, values between 5 and 10 are "okay".
        # Above 10 is too high and so should be removed.
        self.thresh = thresh
        
        # The statsmodel function will fail with NaN values, as such we have to impute them.
        # By default we impute using the median value.
        # This imputation could be taken out and added as part of an sklearn Pipeline.
        if impute:
            self.imputer = Imputer(strategy=impute_strategy)

    def fit(self, X, y=None):
        print('ReduceVIF fit')
        if hasattr(self, 'imputer'):
            self.imputer.fit(X)
        return self

    def transform(self, X, y=None):
        print('ReduceVIF transform')
        columns = X.columns.tolist()
        if hasattr(self, 'imputer'):
            X = pd.DataFrame(self.imputer.transform(X), columns=columns)
        return ReduceVIF.calculate_vif(X, self.thresh)

    @staticmethod
    def calculate_vif(X, thresh=5.0):
        # Taken from https://stats.stackexchange.com/a/253620/53565 and modified
        dropped=True
        while dropped:
            variables = X.columns
            dropped = False
            vif = [variance_inflation_factor(X[variables].values, X.columns.get_loc(var)) for var in X.columns]
            
            max_vif = max(vif)
            if max_vif > thresh:
                maxloc = vif.index(max_vif)
                print(f'Dropping {X.columns[maxloc]} with vif={max_vif}')
                X = X.drop([X.columns.tolist()[maxloc]], axis=1)
                dropped=True
        return X

In [6]:
transformer = ReduceVIF()
X= transformer.fit_transform(X,y);
X.head()

ReduceVIF fit
ReduceVIF transform
Dropping TMIN with vif=89.44873869529434
Dropping Fastest 2 minute wind speed with vif=66.86894742145277
Dropping Average Wind Speed with vif=35.96059634968216
Dropping Fastest 5 second wind speed with vif=34.57579650324814


,PRCP,TMAX,5 second wind speed squared,2 min wind speed squared,Avg Wind Speed Squared,Fog/Ice,Heavy/Freezing Fog,Thunder,Smoke/Haze
0,0.024027,0.638747,0.009206,0.010812,0.034255,0.521739,1.521739e-01,6.521739e-02,1.739130e-01
1,0.000000,0.764706,0.507675,0.011893,0.002474,1.000000,1.000000e+00,2.775558e-17,1.000000e+00
2,0.007368,0.864706,0.206135,0.073630,0.099192,0.600000,2.000000e-01,2.775558e-17,4.000000e-01
3,0.000000,0.764706,0.957100,0.202051,0.017456,0.000000,2.775558e-17,2.775558e-17,2.775558e-17
4,0.150000,0.554902,0.036995,0.087547,0.221798,0.400000,1.387779e-17,2.000000e-01,2.000000e-01


In [7]:
finalData = pd.concat([X,y], axis=1);
finalData.to_csv('Final Data', index=False);

In [8]:
X_new = SelectKBest(chi2, k=5).fit_transform(X.values,y.values)
X = pd.DataFrame(X_new);

In [9]:
X.head()

,0,1,2,3,4
0,0.009206,0.010812,0.521739,1.521739e-01,6.521739e-02
1,0.507675,0.011893,1.000000,1.000000e+00,2.775558e-17
2,0.206135,0.073630,0.600000,2.000000e-01,2.775558e-17
3,0.957100,0.202051,0.000000,2.775558e-17,2.775558e-17
4,0.036995,0.087547,0.400000,1.387779e-17,2.000000e-01


In [10]:
finalData.head()

,PRCP,TMAX,5 second wind speed squared,2 min wind speed squared,Avg Wind Speed Squared,Fog/Ice,Heavy/Freezing Fog,Thunder,Smoke/Haze,Power Outage
0,0.024027,0.638747,0.009206,0.010812,0.034255,0.521739,1.521739e-01,6.521739e-02,1.739130e-01,0.0
1,0.000000,0.764706,0.507675,0.011893,0.002474,1.000000,1.000000e+00,2.775558e-17,1.000000e+00,0.0
2,0.007368,0.864706,0.206135,0.073630,0.099192,0.600000,2.000000e-01,2.775558e-17,4.000000e-01,0.0
3,0.000000,0.764706,0.957100,0.202051,0.017456,0.000000,2.775558e-17,2.775558e-17,2.775558e-17,0.0
4,0.150000,0.554902,0.036995,0.087547,0.221798,0.400000,1.387779e-17,2.000000e-01,2.000000e-01,0.0
